# LJ Tutorial 

The target trajectory files in this example were generated with the code below (hidden, click to reveal):

In [ ]:
# This cell takes 40 minutes. Don't uncomment and rerun it unless you have a good reason.
# trajectory{kT}.gsd files were created here

#for i,kT in enumerate(kTs):
#    hoomd.context.initialize("")
#    system = hoomd.init.create_lattice(
#        unitcell = (hoomd.lattice.sc(a=1.58, type_name="A")), 
#        n = n,
#    )
#    
#    nl = hoomd.md.nlist.cell()
#    lj = hoomd.md.pair.lj(r_cut=2.5, nlist=nl)
#    lj.pair_coeff.set('A', 'A', epsilon=1.0, sigma=1.0)
#    hoomd.md.integrate.mode_standard(dt=0.001)
#    _all = hoomd.group.all()
#    nvt = hoomd.md.integrate.nvt(group=_all, kT=kT, tau=1)
#    nvt.randomize_velocities(seed=23)
#    hoomd.analyze.log(a
#        filename=f'LJ_kT{kT}.log',
#        quantities=["time", "temperature", "potential_energy"],
#        period=100,
#        overwrite=True
#    )
#    hoomd.dump.gsd(f'trajectory{kT}.gsd', period=5e3, group=_all, overwrite=True)
#    hoomd.run(1e6)

# Check that the system is equilibrated.

#fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True)
#fig.suptitle("Potential Energy vs Timestep")
#for i, kT in enumerate(kTs):   
#    log = np.loadtxt(f'state{i}/LJ_kT{kT}.log', skiprows=1)
#    ax1.plot(log[:,0],log[:,3], label=f"kT={kT}")
#    ax2.plot(log[-50:,0],log[-50:,3], label=f"kT={kT}")
#    ax1.set_xlabel("Timestep")
#    ax2.set_xlabel("Timestep")
#    ax1.set_ylabel("Potential Energy (AU)")
#    ax1.legend()
#    ax2.legend()

#for i,kT in enumerate(kTs):
#    gsdfile = f'state{i}/trajectory{kT}.gsd'
#    with gsd.hoomd.open(gsdfile) as t:
#        rdf = freud.density.RDF(bins=101, r_max=5)
#        for snap in t[-50:]:
#            rdf.compute(system=snap, reset=False)
#        data = np.stack((rdf.bin_centers, rdf.rdf)).T
#        np.savetxt(f"rdfs/rdftarget{i}", data)
#        plt.plot(rdf.bin_centers, rdf.rdf)
#        plt.title(f"State {i} kT = {kT}")
#        plt.xlabel("r")
#        plt.ylabel("g(r)")
#        plt.show()

In [1]:
import itertools

import freud
import gsd
import gsd.hoomd
import hoomd
import hoomd.md
import matplotlib.pyplot as plt
import mdtraj as md
import numpy as np

from msibi import MSIBI, State, Pair, mie
from msibi.utils.general import get_msibi_instance

In [2]:
%%bash
rm -rf state*/* rdfs/pair* potentials/* f_fits.log state*/log.txt state*/err.txt state*/run.py state*/query.dcd

In [3]:
kTs = [0.5, 1.5, 2.0]
n = 12
n_particles = n**3

## Create MSIBI instance

In [4]:
# Pass in values that will be used 
opt = MSIBI(
    rdf_cutoff=5.0,
    n_rdf_points=101,
    max_frames=10,
    pot_cutoff=5.0,
    smooth_rdfs=True,
    verbose=True
)

## Specify states

In [5]:
# Each state should have it's own corresponding trajectory (gsd) file
# Create a state by passing in a name, trajectory file, and alpha value
# The MSIBI instance created earlier needs to be passed in as well

stateA = State(name="A",
               kT=0.5,
               traj_file="trajectory0.5.gsd",
               optimizer = opt,
               alpha=0.25
              )
stateB = State(name="B",
               kT=1.5,
               traj_file="trajectory1.5.gsd",
               optimizer = opt,
               alpha=0.50
              )
stateC = State(name="C",
               kT=2.0,
               traj_file="trajectory2.0.gsd",
               optimizer = opt,
               alpha=0.25
              )

OSError: No such file: /Users/jenny/Projects/msibi/msibi/tutorials/lj/trajectory0.5.gsd

## Specify pairs

In [ ]:
indices = list(itertools.combinations(range(n_particles), 2))  

# Each type of possible pair-wise interaction of the system should have its own Pair() created
# Each Pair() object needs to have the states created above added

initial_guess = mie(opt.pot_r, 1.0, 1.0)
pair0 = Pair(type1="A", type2="A", potential=initial_guess)

pair0.add_state(stateA,
                calculate_target_rdf=True,
                pair_indices=indices
               )
pair0.add_state(stateB,
                calculate_target_rdf=True,
               pair_indices=indices
               )
pair0.add_state(stateC,
                calculate_target_rdf=True,
               pair_indices=indices
               )

In [ ]:
#pair0.compute_current_rdf(state=stateA, smooth=False)
#plt.plot(pair0.states[stateA]['current_rdf'][:, 0], pair0.states[stateA]['current_rdf'][:, 1])

## Implement MSIBI algorithm

In [ ]:
# Run the optimization steps
# Pass in lists of any state and pair objects created above

opt.optimize(states=[stateA, stateB, stateC],
             pairs=[pair0],
             n_iterations=5,
             engine='hoomd'
            )

## Plot results

In [ ]:
for state in states:
    plt.title(f'{state.name} kt={state.kT}')
    
    for step in range(0,opt.n_iterations):
        try:
            step_rdf = np.loadtxt(f'rdfs/pair_A-A-state_{state.name}-step{step}.txt')
            plt.plot(step_rdf[:,0],step_rdf[:,1], label=f'step {step}')
        except OSError:
            break
    target = np.loadtxt(f'rdfs/rdf.target{state.name.strip("state")}.t1t1.txt')
    plt.plot(target[:,0], target[:,1], label='target')
    plt.legend()
    plt.show()

In [ ]:
for step in range(0, opt.n_iterations):
    plt.ylim([-1.5,0.5])
    plt.xlim([0.5,2])
    plt.title("MSIBI potentials by iteration")
    try:
        step_pot = np.loadtxt(f'potentials/step{step}.pot.A-A.txt')
        plt.plot(step_pot[:,0],step_pot[:,1], label=f'step {step}')
    except OSError:
        break
plt.legend()
plt.show()